<a href="https://colab.research.google.com/github/splendiferousnoctifer/naturalTouchTex/blob/main/senseTouch_machineLearning_sensorPattern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comparison of different Sensor-Patterns
Trial of different Sensorpatterns for optimisation od Sensor placement and amount.

In [ ]:
#@markdown Use real data:
real_data = True #@param {type:"boolean"}

#@markdown Data creation parameters:

sample_number = 1000 #@param {type:"number"}
data_point_dimension = 7  #@param {type: "number"}

add_noise = True #@param {type:"boolean"}
noise_amount = 0.01 #@param {type: "number"}

#@markdown Train-Test Split parameters:

test_size_split = 0.33 #@param {type:"number"}
data_split_shuffle = True #@param {type:"boolean"}

# Setup

## Imports

In [ ]:
import random
import numpy as np
import pandas as pd
from matplotlib import cm
import matplotlib.pyplot as plt 
from sklearn import linear_model
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from scipy.interpolate import interp2d, SmoothBivariateSpline, RectBivariateSpline

np.set_printoptions(suppress=True) # don't use scientific notation
PATH = '/content/drive/MyDrive/mil/data/analytics/'


## Data

In [ ]:
w, h = 1,1
samples = sample_number

p =  []
for i in range(samples):
  mod_x = random.randint(1,99) / 100
  mod_y = random.randint(10,90) / 100
  mod_z = random.randint(-100,100) / 10000

  p.append((w * mod_x, h * mod_y, 0.2 + mod_z)) # press position and depth in centimeters

p.append((0.5, 0.5, 0)) #no indent for delta calculation

p = np.asarray(p)

In [ ]:
z_data = []

N = data_point_dimension # data points in each dimension
x = np.linspace(0,w,N)
y = np.linspace(0,h,N)

for i in range(samples + 1):
  cond = np.zeros((3,3))
  cond[1,1] = p[i,2]

  f = interp2d([0, p[i,0], w], [0, p[i,1], h], cond, kind='linear')
  z_data.append(f( x, y ))

x_data = np.asarray([[x]*N]*(samples+1))
y_data = np.asarray([np.asarray([y]*N).T] * (samples+1))
z_data = np.asarray(z_data)


**Flat Textile**

In [ ]:
fig = go.Figure(
    data=[
          go.Scatter3d(x=x_data[-1].flatten(), y=y_data[-1].flatten(), z=z_data[-1].flatten())#, mode='markers', marker=dict(size=5, color='red')),
        ]
)


scene = dict(
  xaxis = dict(nticks=4, range=[-0.1,1.1],),
  yaxis = dict(nticks=4, range=[-0.1,1.1],),
  zaxis = dict(nticks=4, range=[-1,1],)
)

fig.update_layout()# scene = scene)

fig.show()

In [ ]:
z_data[np.where(np.isnan(z_data))] = 0 #in some cases 0 gets interpreted as nan

Merge individual axes into one point data set:

In [ ]:
synthetic_data = np.zeros((samples+1,N,N,3))
synthetic_data[:,:,:,0] = x_data
synthetic_data[:,:,:,1] = y_data
synthetic_data[:,:,:,2] = z_data

**Add noise**

In [ ]:
if add_noise:
  mu, sigma = 0, w/(N-1) * noise_amount/100
  noise = np.random.normal(mu, sigma, synthetic_data.shape) 
  synthetic_data = synthetic_data + noise

**Data decision**

In [ ]:
if real_data:
  data = np.load(PATH + 'data_norm_rot.npy')
  peaks = np.load(PATH + 'peaks_norm.npy')
  flat_index = 21845
else:
  data = synthetic_data
  peaks = np.arange(data.shape[0])
  flat_index = -1

## Sensor Delta Calculation

In [ ]:
points = np.zeros((data.shape[0], 49,3))
for i in range(points.shape[0]):
  points[i] = np.reshape(data[i], (49,3))

In [ ]:
fig = go.Figure(
    data=[
          go.Scatter3d(x=points[flat_index,:,0], y=points[flat_index,:,1], z=points[flat_index,:,2])#, mode='markers', marker=dict(size=5, color='red')),
        ]
)


scene = dict(
  xaxis = dict(nticks=4, range=[-0.1,1.1],),
  yaxis = dict(nticks=4, range=[-0.1,1.1],),
  zaxis = dict(nticks=4, range=[-1,1],)
)

fig.update_layout( scene = scene)

fig.show()

### Sensors

In [ ]:
sensors = [
  [42,35], [42,34], [43,34], [44,37], [44,38], [45,38], [46,38], [46,39], [47,40], [48,40], [48,41], #sensor 11, top row
  [48,47], [41,40], [34,33], [34,26], [27,26], [20,26], [20,19], [13,12], [13,5], [6,5], #sensor 21, right column
  [6,13], [5,12], [4,11], [4,10], [3,10], [2,10], [2,9], [1,8], [0,8], [0,7], #sensor 31, bottom row
  [0,1], [7,8], [14,15], [14,22], [21,22], [28,22], [28,29], [35,36], [42,43], #sensor 40, left column
  [36,30], [40,32], [12,18], [8,16], #sensor 44, circle
  [23,24], [31,24], [25,24], [17,24] #sensor 45, cross
]
sensors = np.asarray(sensors)

In [ ]:
distance_flat, distance_indent = [], []
frame = points[flat_index,:,:]

for i in range(sensors.shape[0]):
  distance_flat.append(
      np.asarray([np.linalg.norm(frame[sensors[i][0],:] - frame[sensors[i][1],:])] * (data.shape[0]-1))
  )
  distance_indent.append([])

  for j in range(data.shape[0] - 1):
    distance_indent[i].append(
        np.asarray(np.linalg.norm(points[j,sensors[i][0],:] - points[j,sensors[i][1],:]))
    )

distance_flat = np.asarray(distance_flat)
distance_indent = np.asarray(distance_indent)

### Delta

In [ ]:
delta = distance_indent - distance_flat

#delta[np.where(delta==0)] = 0.00000000000001
delta_inverse = np.divide(1,delta)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning:

divide by zero encountered in true_divide



## Pattern Configurations

In [ ]:
test_config = np.array((
  #3 Sensors
  [5,35,15], 
  [5,29,19],

  #4 Sensors
  [5,15,25,29],  
  [1,9,19,29],  
  [44,45,46,47],

  #6 Sensors
  [1,7,13,19,27,33],  
  [5,12,18,25,32,38],

  #8 Sensors
  [5,41,15,42,25,43,29,40],  
  [1,5,9,15,19,25,29,35], 

  #12 Sensors
  [1,4,6,9,14,16,19,24,26,29,34,36],  
  [1,3,7,9,13,17,19,23,27,29,33,37],
  [2,5,8,12,15,18,22,25,28,32,29,38],  
  [1,2,8,9,12,18,19,22,28,29,32,38],
  [4,6,14,16,24,26,34,36,40,41,42,43],

  # 16 Sensors
  [0,39,3,7,11,10,13,17,21,20,23,27,31,30,33,37],  
  [0,39,5,11,10,15,21,20,25,31,30,35,40,41,42,43],        
  [1,3,5,7,9,13,15,17,19,23,25,27,29,33,35,37],

  # 20 Sensors
  [1,3,4,6,7,9,13,14,16,17,19,23,24,26,27,29,33,34,36,37],  
  [2,3,5,7,8,12,13,15,17,18,22,23,25,27,28,32,33,35,37,38],

  # 24 Sensors
  [1,2,3,5,7,8,9,12,13,15,17,18,19,22,23,25,27,28,29,32,33,35,37,38],
  [38,2,3,4,6,7,8,12,13,14,16,17,18,22,23,24,26,27,28,32,33,34,36,37],

  # 28 Sensors
  [0,2,3,5,7,8,10,11,12,13,15,17,18,20,21,22,23,25,27,28,30,31,32,33,35,37,38,39],

  # 32 Sensors
  [0,1,2,3,5,7,8,9,10,11,12,13,15,17,18,19,21,20,22,23,25,27,28,29,30,31,32,33,35,37,38,39],
  [0,2,3,5,7,8,10,11,12,13,15,17,18,20,21,22,23,25,27,28,30,31,32,33,35,37,38,39,40,41,42,43],
  [0,2,3,5,7,8,10,11,12,13,15,17,18,20,21,22,23,25,27,28,30,31,32,33,35,37,38,39,44,45,46,47],

  # 36 Sensors
  [0,1,2,3,5,7,8,9,10,11,12,13,15,17,18,19,20,21,22,23,25,27,28,29,30,31,32,33,35,37,38,39,44,45,46,47]

), dtype = object)

## Press Points

Real Data

In [ ]:
x,y,z = [],[],[]
for frame in data[peaks]:
  x.append(frame[:,:,0].flatten())
  y.append(frame[:,:,1].flatten())
  z.append(frame[:,:,2].flatten())


x,y,z = np.asarray(x), np.asarray(y), np.asarray(z)

z_maxs = []
for zframe in z:
  z_maxs.append(np.amax(zframe))

z_maxs = np.asarray(z_maxs)

peak_points = []
for i in range(peaks.shape[0]):
  loc = np.where(z[i] == z_maxs[i])
  #print(x[i,loc])
  #print(x[loc[0]])
  peak_points.append((x[i,loc], y[i,loc], z[i,loc]))

peak_points = np.asarray(peak_points)

indent = np.zeros((peaks.shape[0], 3))

for i in range(peaks.shape[0]):
  indent[i] = peak_points[i].flatten()

Assign presspoints according to mode

In [ ]:
if real_data:
  point_x = np.asarray(indent[:,0])
  point_y = np.asarray(indent[:,1])
  point_z = np.asarray(indent[:,2])
else:
  point_x = np.asarray(p[:-1,0])
  point_y = np.asarray(p[:-1,1])
  point_z = np.asarray(p[:-1,2])

In [ ]:
print(point_y.shape)

(1200,)


# ML

##Models

###Random Forest

In [ ]:
def rndForest(training_x, training_y, test_x, test_y):
  rf = RandomForestRegressor()
  rf.fit(training_x, training_y)

  yhat = rf.predict(test_x)

  #print('Random Forest:')
  #print('   Score:', r2_score(test_y, yhat))
  #print('   Mean Absolute Error:', mean_absolute_error(test_y, yhat))
  #print('   Mean Squared Error:', np.sqrt(mean_squared_error(test_y, yhat)))

  return(np.asarray((r2_score(test_y, yhat), mean_absolute_error(test_y, yhat), np.sqrt(mean_squared_error(test_y, yhat)))))

**Andere Parameter**


In [ ]:
def rndForestParam(training_x, training_y, test_x, test_y, estimators = 200, depth = None, tuning = False):
  clf = RandomForestRegressor(n_estimators=estimators,  max_depth=depth, random_state=0)
  clf.fit(training_x, training_y)

  yhat = clf.predict(test_x)

  #print('Random Forest /w Parameter:')
  #print('   Score:', r2_score(test_y, yhat))
  #print('   Mean Absolute Error:', mean_absolute_error(test_y, yhat))
  #print('   Mean Squared Error :', np.sqrt(mean_squared_error(test_y, yhat)))

  #region tuning
  if tuning:
    param_grid = { 'n_estimators': [10, 50, 100, 200], 'max_depth': [1,2,3,5,10, None]}
    search = GridSearchCV(clf, param_grid, cv=5)
    search.fit(training_x, training_y)
    print(search.best_params_)
    search.score(training_x, training_y)
  #endregion

  return(np.asarray((r2_score(test_y, yhat), mean_absolute_error(test_y, yhat), np.sqrt(mean_squared_error(test_y, yhat)))))

**Hyperparameter Tuning**

See method above under tuning.

**Plot**

In [ ]:
def plotRndForest():
  rf_prediction = np.zeros((samples, 3))
  for i in range(samples):
    rf_prediction[i] = np.asarray(rf.predict([X[i,:]])).flatten()


  error = p[:-1] - rf_prediction

  err_dist = np.zeros(error.shape[0])
  for i in range(error.shape[0]):
    err_dist[i] = np.linalg.norm(error[i])

  #print('Average Distance Error =', np.average(err_dist) )

  plt.figure(figsize=(20,5))
  plt.plot(err_dist)
  plt.show()

### Polynomial Regression

In [ ]:
def polyReg(training_x, training_y, test_x, test_y, degree = 7, tuning = False):
  degree=7
  polyreg=make_pipeline(PolynomialFeatures(degree),LinearRegression())
  polyreg.fit(training_x, training_y)

  yhat = polyreg.predict(test_x)

  #print('Polynomial Regression:')
  #print('   Score:', r2_score(test_y, yhat))
  #print('   Mean Absolute Error:', mean_absolute_error(test_y, yhat))
  #print('   Mean Squared Error :', np.sqrt(mean_squared_error(test_y, yhat)))

  #region tuning
  if tuning:
    param_grid = { 'polynomialfeatures__degree': [1,2,3,5,10]}
    search = GridSearchCV(polyreg, param_grid, cv=5)
    search.fit(X_train, y_train)
    print(search.best_params_)
    search.score(X_train, y_train)
  #endregion

  return(np.asarray((r2_score(test_y, yhat), mean_absolute_error(test_y, yhat), np.sqrt(mean_squared_error(test_y, yhat)))))

**Hyperparameter Tuning**

See method above for tuning.

## Split Data & Run models

In [ ]:
test_results = np.zeros((test_config.shape[0], 4, 3))

In [ ]:
print(test_results.shape)

(26, 4, 3)


In [ ]:
for i in range(17,18):
  #TODO

  X = np.swapaxes(delta[test_config[i]], 0,1)
  if real_data:
    X =  X[peaks]
  Y = np.swapaxes(np.asarray((point_x, point_y, point_z)),0,1)

  X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size_split, shuffle=data_split_shuffle)

  print('Pattern:', i+1)
  #print('Sensor amount:', np.asarray(test_config[i]).shape[0])

  test_results[i] = np.asarray((
      np.asarray((i+1, np.asarray(test_config[i]).shape[0],0)),
      rndForest(X_train, y_train, X_test, y_test),
      rndForestParam(X_train, y_train, X_test, y_test),
      polyReg(X_train, y_train, X_test, y_test))
  )

Pattern: 18


## Eval

In [ ]:
print(test_results)